In [41]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
import numpy as np

In [42]:
x_train = pd.read_csv('bank_x_train.csv')
y_train = pd.read_csv('bank_y_train.csv')
x_test = pd.read_csv('bank_x_test.csv')
y_test = pd.read_csv('bank_y_test.csv')

In [43]:
# 불필요한 컬럼삭제
del x_train['Unnamed: 0']
del y_train['Unnamed: 0']
del x_test['Unnamed: 0']
del y_test['Unnamed: 0']

In [44]:
x_train

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001032,Male,No,0,Graduate,No,4950,0.0,125,360,1,Urban
1,LP001824,Male,Yes,1,Graduate,No,2882,1843.0,123,480,1,Semiurban
2,LP002928,Male,Yes,0,Graduate,No,3000,3416.0,56,180,1,Semiurban
3,LP001814,Male,Yes,2,Graduate,No,9703,0.0,112,360,1,Urban
4,LP002244,Male,Yes,0,Graduate,No,2333,2417.0,136,360,1,Urban
...,...,...,...,...,...,...,...,...,...,...,...,...
379,LP002585,Male,Yes,0,Graduate,No,3597,2157.0,119,360,0,Rural
380,LP001841,Male,No,0,Not Graduate,Yes,2583,2167.0,104,360,1,Rural
381,LP002820,Male,Yes,0,Graduate,No,5923,2054.0,211,360,1,Rural
382,LP001744,Male,No,0,Graduate,No,2971,2791.0,144,360,1,Semiurban


In [45]:
x_train = x_train.drop(['Loan_ID'], axis=1)
x_test = x_test.drop(['Loan_ID'],axis=1)

### 원핫인코딩

In [46]:
transformer = make_column_transformer((OneHotEncoder(), ['Gender','Married','Dependents','Education','Self_Employed', 'Property_Area']), remainder='passthrough')

In [47]:
transformer.fit(x_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder', OneHotEncoder(),
                                 ['Gender', 'Married', 'Dependents',
                                  'Education', 'Self_Employed',
                                  'Property_Area'])])

In [48]:
x_train = transformer.transform(x_train)
x_test = transformer.transform(x_test)

### 레이블 인코딩

In [49]:
le = LabelEncoder()

In [50]:
le.fit(y_train)

C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LabelEncoder()

In [51]:
labels = ['비승인','승인']

In [52]:
y_train = le.transform(y_train)
y_test = le.transform(y_test)

C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [55]:
x_train

array([[  0.,   1.,   1., ..., 125., 360.,   1.],
       [  0.,   1.,   0., ..., 123., 480.,   1.],
       [  0.,   1.,   0., ...,  56., 180.,   1.],
       ...,
       [  0.,   1.,   0., ..., 211., 360.,   1.],
       [  0.,   1.,   1., ..., 144., 360.,   1.],
       [  0.,   1.,   0., ..., 255., 360.,   1.]])

In [56]:
y_test

array([0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 1])

### EDA

### 5-7 모델링(모델학습, 예측, 평가)

In [57]:
model = VotingClassifier(estimators = [('lr', LogisticRegression()), #estimators:하이퍼파라미터, 어떤 모델쓸지 불러오고, 디폴트값은 하드보팅임
                                       ('kn', KNeighborsClassifier()),
                                       ('dt', DecisionTreeClassifier())])

In [58]:
model.fit(x_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('kn', KNeighborsClassifier()),
                             ('dt', DecisionTreeClassifier())])

In [59]:
model.score(x_train, y_train)

0.8932291666666666

In [60]:
model.score(x_test, y_test)

0.7291666666666666

### 새로운 값으로 예측

In [65]:
x_test = [['Male', 'Yes', '0', 'Graduate', 'No', 14583, 0, 436, 360, 1,'Semiurban']]
x_test = pd.DataFrame(x_test, columns =  ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'ApplicantIncome', 
          'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area'])
x_test = transformer.transform(x_test)

In [66]:
y_predict = model.predict(x_test)

In [70]:
labels[y_predict[0]]

'승인'